In [15]:
import cv2
import os
import numpy as np

In [13]:
imagesPath = os.listdir('bbox')
imagesPath.remove('.DS_Store')

In [63]:
imList = np.zeros((0, 100, 100, 3))
y_label = []
for file in imagesPath:
    im = cv2.imread('bbox/' + file)
    im = cv2.resize(im, (100,100))
    imList = np.insert(imList, 0, values=np.array([im]), axis=0)
    y_label.append(0)
    
    im = cv2.flip(im,1)
    imList = np.insert(imList, 0, values=np.array([im]), axis=0)
    y_label.append(1)
y_label = np.array(y_label)

In [74]:
import numpy as np  
np.random.seed(1337)  # for reproducibility   
from keras.models import Sequential  
from keras.layers import Dense, Dropout, Activation, Flatten  
from keras.layers import Convolution2D, MaxPooling2D  
from keras.utils import np_utils  
from keras import backend as K  
   
batch_size = 128  
nb_classes = 2
epochs = 12
# input image dimensions  
img_rows, img_cols = 100, 100
# number of convolutional filters to use  
nb_filters = 32  
# size of pooling area for max pooling  
pool_size = (2, 2)  
# convolution kernel size  
kernel_size = (3, 3)  
  
X_train = imList
X_test = imList
y_train = y_label
y_test = y_label
input_shape = (100,100,3)

X_train = X_train.astype('float32')  
X_test = X_test.astype('float32')  
X_train /= 255  
X_test /= 255  
X_train = X_train.astype('float32') 
print('X_train shape:', X_train.shape)  
print(X_train.shape[0], 'train samples')  
print(X_test.shape[0], 'test samples')  
  
# label => onehot-form
Y_train = np_utils.to_categorical(y_train, nb_classes)  
Y_test = np_utils.to_categorical(y_test, nb_classes)  
  
# build model  
model = Sequential()  
  
# conv1
model.add(Convolution2D(nb_filters, (kernel_size[0], kernel_size[1]),  
                        padding='same',  
                        input_shape=input_shape))   
model.add(Activation('relu'))

# ============
# conv2
model.add(Convolution2D(nb_filters, (kernel_size[0], kernel_size[1]))) 
model.add(Activation('relu'))
# max-pool 
model.add(MaxPooling2D(pool_size=pool_size)) 
model.add(Dropout(0.25)) 

# ============
model.add(Flatten())
# fc1  
model.add(Dense(128)) 
model.add(Activation('relu')) 
model.add(Dropout(0.5))

# ============
# fc2  
model.add(Dense(nb_classes)) 
# Softmax layer
model.add(Activation('softmax')) 
  
# compile model  
model.compile(loss='categorical_crossentropy',  
              optimizer='adadelta',  
              metrics=['accuracy'])  
# train model  
model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs,  
          verbose=1, validation_data=(X_test, Y_test))  
# evaluate model  
score = model.evaluate(X_test, Y_test, verbose=0)  
print('Test score:', score[0])  
print('Test accuracy:', score[1])  

X_train shape: (488, 100, 100, 3)
488 train samples
488 test samples
Train on 488 samples, validate on 488 samples
Epoch 1/12
488/488 [==============================] - 10s 21ms/step - loss: 3.9821 - acc: 0.5492 - val_loss: 0.6709 - val_acc: 0.5000
Epoch 2/12
488/488 [==============================] - 9s 19ms/step - loss: 0.5844 - acc: 0.7418 - val_loss: 0.3548 - val_acc: 0.9713
Epoch 3/12
488/488 [==============================] - 9s 19ms/step - loss: 0.8280 - acc: 0.8053 - val_loss: 0.2405 - val_acc: 0.9754
Epoch 4/12
488/488 [==============================] - 9s 19ms/step - loss: 0.7599 - acc: 0.6148 - val_loss: 0.5652 - val_acc: 0.8770
Epoch 5/12
488/488 [==============================] - 9s 19ms/step - loss: 0.3285 - acc: 0.9303 - val_loss: 0.3827 - val_acc: 0.7418
Epoch 6/12
488/488 [==============================] - 9s 19ms/step - loss: 0.2653 - acc: 0.9160 - val_loss: 0.1196 - val_acc: 0.9918
Epoch 7/12
488/488 [==============================] - 9s 19ms/step - loss: 0.0755 - ac

In [81]:
# save model
model_json = model.to_json()
model.save_weights('weights.h5')
import json
with open("model.json",'w') as json_file:
    json_file.write(model_json)